# Capítulo 11 

# Chapter 11. Training Deep Neural Networks

En el Capítulo 10 presentamos las redes neuronales artificiales y entrenamos nuestras primeras redes neuronales profundas.

Pero eran redes poco profundas, con solo unas pocas capas ocultas.

¿Qué sucede si necesita abordar un problema complejo, como detectar cientos de tipos de objetos en imágenes de alta resolución?

Es posible que deba entrenar un DNN mucho más profundo, tal vez con 10 capas o muchas más, cada una con cientos de neuronas, unidas por cientos de miles de conexiones.

Training a deep DNN isn’t a walk in the park. Here are some of the problems you could run into:

You may be faced with the tricky vanishing gradients problem or the related exploding gradients problem. 

This is when the gradients grow smaller and smaller, or larger and larger, when flowing backward through the DNN during training. 

Both of these problems make lower layers very hard to train.

You might not have enough training data for such a large network, or it might be too costly to label.

Training may be extremely slow.

A model with millions of parameters would severely risk overfitting the training set, especially if there are not enough training instances or if they are too noisy.

In this chapter we will go through each of these problems and present techniques to solve them. 

We will start by exploring the vanishing and exploding gradients problems and some of their most popular solutions.

Next, we will look at transfer learning and unsupervised pretraining, which can help you tackle complex tasks even when you have little labeled data.

Then we will discuss various optimizers that can speed up training large models tremendously. 

Finally, we will go through a few popular regularization techniques for large neural networks.

With these tools, you will be able to train very deep nets. 

Welcome to Deep Learning!